In [1]:
from numpy.core.fromnumeric import shape
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch import nn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score